In [ ]:
import os
os.chdir('..')

In [ ]:
import sys
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import math
import re
import pulp
from scipy.stats import entropy
import scipy as sp

import warnings

from acpd_utils import check_results
from acpd_utils import validate_T_matrix
from acpd_methods import generate_T_method_3

np.random.seed(10)

# Statistical tests

In [ ]:
#Returns True if all the values in 'x' are integers (even if they have float format) 
def assert_int_array(x):
    return np.all(np.abs(np.round(x)-x)<1e-8)


#Given an array of probabilities 'probs' and a number of samples, returns the absolute frequencies 
#corresponding to probs*num_samples. 
#2 possible control levels: 
#   expect_int_freqs=True:  stops if the frequency vector is not an integer array
#   expect_int_freqs=False: the computed frequencies are rounded in order to be integers 
#                           (and a warning message is raised)
def probs2freqs(probs, num_samples, expect_int_freqs):
    #Multiply the probabilities by the number of samples
    probs_samples = np.copy(probs*num_samples)
    
    #Check if the frequencies are integers (if not, either raise an error or round the values):
    if not assert_int_array(probs_samples):
        if expect_int_freqs:
            sys.exit("probs*num_samples is not an Int array (freqs)") #Return an error
        else:
            warnings.warn("Freqs=probs*num_samples is not an Int array")
            warnings.warn("Rounding will be applied to the frequencies")
            probs_samples = np.round(probs_samples) #Round the frequencies
            
    assert assert_int_array(probs_samples) #Final sanity check
    
    #Return the vector with the absolute frequencies
    return probs_samples


#Given an array of frequencies 'freqs' (obtained by multiplying a vector of probabilities with a number of samples),
#correct the absolute frequencies (if required) in order to ensure that the sum matches that number of samples.
def correct_rounded_freqs(freqs, num_samples):
    out_freqs = np.copy(freqs)
    #Compute difference between total frequencies and the expected number of samples
    tmp_dif = int(np.abs(np.sum(out_freqs) - num_samples))
    #Fix the frequencies:
    if np.sum(out_freqs)>num_samples:  
        out_freqs[np.argmax(out_freqs)] -= tmp_dif #decrease from the element with max. frequency
    else:
        out_freqs[np.argmin(out_freqs)] += tmp_dif #increase from the element with min. frequency
    #Sanity checks
    assert np.sum(out_freqs)==num_samples, "Frequencies still wrong after correction: %d" % np.sum(out_freqs)
    assert np.all(out_freqs>=0) and np.all(out_freqs<=num_samples), "Out of bound values in frequencies"
    assert assert_int_array(out_freqs)
    
    return out_freqs



#TWO SAMPLE TESTS
#Evaluates a specified two sample statistical test (categorical variables) for the label-shift detector, 
#and returns the p-value.
# @input labels:       labels/categories of the problem
# @input num_samples:  number of elements in the 'samples'
# @input py_exp:       'expected' probability distribution
# @input py_obs:       'observed' probability distribution
# @input test_method:  statistical test to be emploted. Supported option: "chisq_cont" (Chi-Square test)
# @output:             p-value of the statistical test
def evaluate_test(labels, num_samples, py_exp, py_obs, test_method):
    #Convert the probabilities into absolute frequencies
    py_exp_samples = probs2freqs(py_exp, num_samples, expect_int_freqs=True)
    py_obs_samples = probs2freqs(py_obs, num_samples, expect_int_freqs=False)
    
    #Convert to integer arrays
    py_exp_samples = np.array(py_exp_samples, dtype=np.int64)
    py_obs_samples = np.array(py_obs_samples, dtype=np.int64)
    
    #Sanity checks
    if np.sum(py_exp_samples)!=num_samples: 
        sys.exit("Total frequency of py_exp (%d) does not match num_samples (%d)" % (np.sum(py_exp_samples), num_samples))
    if np.sum(py_obs_samples)!=num_samples: 
        warnings.warn("Total frequency of py_obs (%d) does not match num_samples (%d)" % (np.sum(py_obs_samples), num_samples))
        warnings.warn("Total frequency of py_obs will be corrected.")
        py_obs_samples = correct_rounded_freqs(py_obs_samples, num_samples) #Correct the frequencies
    assert np.sum(py_exp_samples)==np.sum(py_obs_samples), "The total frequency of the two vectors is not the same"
    assert np.sum(py_obs_samples)==num_samples, "Frequencies do not match expected"
    assert len(labels)==len(py_exp) and len(labels)==len(py_obs), "Wrong number of categories"
    
    #Run the statistical test    
    if test_method=="chisq_cont":
        statistic, pvalue, cont_dof, cont_expec = sp.stats.chi2_contingency(np.array([py_exp_samples,py_obs_samples]))
    else:
        sys.exit("Statistical test '%s' not supported!" % test_method)
        
    return pvalue

In [ ]:
#Auxiliary function to adjust a target distribution so that it is not significantly different from a source
#distribution, according to a statistical test (specified by the 'test_method' argument).
#Given a reference distribution 'p_y_init' and a probability distribution 'p_y_target_signif',
#finds and returns another distribution 'p_y_target',
#  p_y_target = (1-\alpha)*p_y_init + (\alpha)*p_y_target_signif
#by finding which is the maximum value of \alpha so that there are not significant differences between
#p_y_target and p_y_init, according to a selected statistical test (specified by the 'test_method' argument).
def make_py_target_not_significant(p_y_init, p_y_target_signif,
                                   py_test_tol, py_test_width,
                                   eval_n_samp, labels, test_method):
    
    iters = 0 #number of binary-search iterations
    
    #Evaluate if the target distribution is significantly different to p_y_init
    p_y_target = np.copy(p_y_target_signif)
    pvalue = evaluate_test(labels, eval_n_samp, py_exp=p_y_init, py_obs=p_y_target, test_method=test_method)
    
    if pvalue>=(py_test_tol+py_test_width):
        py_alpha_ratio = 1 #Trade-off ratio between the initial distribution and the target distribution
        print("The provided distribution is not detected as significantly different from P(Y)")
        warnings.warn("The provided distribution is not detected as significantly different from P(Y)")
    else:
        py_alpha_ratio = -1 #Trade-off ratio (initialized as negative value)
        
        #Binary search to find the maximum "alpha" parameter for which the target distribution is not detected
        #as a significant change by the selected statistical test.
        l_alpha = 0.0
        r_alpha = 1.0
        while r_alpha-l_alpha > 1e-8:
            mid_alpha = (l_alpha+r_alpha)/2.0
            mid_p_y   = (1-mid_alpha)*p_y_init + mid_alpha*p_y_target_signif

            #Sanity checks
            assert np.abs(np.sum(mid_p_y)-1.0)<1e-8, "mid_p_y  is not a prob. distr."
            assert np.all(mid_p_y>0.0) and np.all(mid_p_y<1.0), "mid_p_y out  of [0,1]"
            
            #Evaluate the test
            pvalue = evaluate_test(labels, eval_n_samp, py_exp=p_y_init, py_obs=mid_p_y, test_method=test_method)
            #Adjust the search
            if pvalue <= (py_test_tol+py_test_width):
                r_alpha = mid_alpha #significant --> reduce alpha (more similar to p_y_init) 
            else:
                l_alpha = mid_alpha #non-significant --> increase alpha (more similar to p_y_target_signif)
                p_y_target = np.copy(mid_p_y) #set the current distribution as the target 
                assert py_alpha_ratio<mid_alpha  #Sanity check
                py_alpha_ratio = np.copy(mid_alpha) #update the ratio
            iters+=1
        
        assert py_alpha_ratio>0.0 and py_alpha_ratio<1.0
    
    
    #Compute the final p-value
    pvalue = evaluate_test(labels, eval_n_samp, py_exp=p_y_init, py_obs=p_y_target, test_method=test_method)
    #Print some details
    print(">> Iterations required:", iters)
    print(">> Balance-Ratio", py_alpha_ratio)
    print(">> Final p-value:", pvalue)
        
    #Sanity checks
    assert np.abs(np.sum(p_y_target)-1.0)<1e-8, "p_y_target_weighted  is not a prob. distr."
    assert np.all(p_y_target>0.0) and np.all(p_y_target<1.0), "p_y_target_weighted not in [0,1]"
    assert pvalue>=(py_test_tol+py_test_width), "Final distribution is significantly different"
    
    return p_y_target

# Global configuration

In [ ]:
#Root path of the project
root_path = "../"

In [ ]:
#Select the dataset
sel_dataset = "emotion" #Tweet Emotion Classification 
print("Main setup: \t " + sel_dataset)

In [ ]:
#Auxiliary function to get the model "ID" (abbreviated identifier) given the "HuggingFaces tag"
#The ID is only used for an easier saving/loading process
def text_model_tag2id(main_setup, model_tag, base_path):
    load_filename = base_path + "models/" + "%s_tag2id_dict.npy" % main_setup  # Filename
    model_tag2id  = np.load(load_filename, allow_pickle=True)[()]  # Load the dictionary which maps tags to IDs
    assert model_tag in model_tag2id, "Tag not found in the dictionary: %s" % model_tag #Sanity check
    return model_tag2id[model_tag] #Return the ID

In [ ]:
#Select a target model (from bhadresh-savani - HuggingFace repository)
model_tag = "bert-base-uncased-emotion"  #** The model used for the experiments
model_id  = text_model_tag2id(sel_dataset, model_tag, root_path)  # model ID (abbreviated identifier)
print("Model tag: %s (id: %s)" % (model_tag, model_id))

In [ ]:
#Load the problem labels
labels_file_path = root_path + "datasets/%s_dataset_train/%s_labels.txt" % (sel_dataset, sel_dataset)
labels = open(labels_file_path, 'r').readlines()
labels = [s.rstrip() for s in labels]
print("Labels", labels)

num_classes = len(labels) #Number of classes

In [ ]:
#Select an adversarial attack
sel_attack = "genetic"  #Genetic algorithm-based word substitution
print("Sel attack: %s" % sel_attack)

In [ ]:
#Select a source distribution:
p_y_init_idx = 0   #Options: [0,1,2]

#In the evaluation we considered three different source distributions P(Y):
if   p_y_init_idx==0:  p_y_init = np.array([0.17, 0.17, 0.17, 0.17, 0.17, 0.15]) #Roughly uniform
elif p_y_init_idx==1:  p_y_init = np.array([0.15, 0.25, 0.15, 0.15, 0.15, 0.15]) #Tweak-2
elif p_y_init_idx==2:  p_y_init = np.array([0.15, 0.15, 0.15, 0.25, 0.15, 0.15]) #Tweak-4
else: sys.exit("Invalid identifier for the source distribution: %d" % p_y_init_idx)
    
#Sanity checks
assert np.abs(np.sum(p_y_init)-1.0)<=1e-8, np.sum(p_y_init)
assert np.all(p_y_init>=0.0) and np.all(p_y_init<=1.0)

print("Source distribution (index %d):" % p_y_init_idx)
print(p_y_init)

In [ ]:
#Statistical test for the label-shift detector (BBSD)
selected_stat_test = "chisq_cont"  #Chi-Square test
print("Selected statistical test: %s" % selected_stat_test)

In [ ]:
#Select the tolerance for the statistical test of the label-shift detector
py_test_tol   = 0.00001 
py_test_width = 0.0001  #Width parameter for the tolerance (to be used during the sampling procedure)

print("Stat. test tolerance: %s + width: %s" % (str(py_test_tol), str(py_test_width)))

In [ ]:
#Set the path in which the results will be saved (if save_path is None, the results will not be saved)
#save_path = None
save_path = root_path + "optimization/label_shift/%s/%s/" % (sel_dataset, sel_attack)

if not (save_path is None):
    print("Save path: %s" % save_path)
else:
    print("Results will not be saved")

# Data to be used in the experimental evaluation

In [ ]:
#Load the sampling considered for the experiments (selected dataset indices and ground-truth labels)
experiment_folder = root_path + "adv_attacks/text/experiments/"
full_filenames    = np.load(experiment_folder + "%s_%s_full_sampling.npy"%(sel_dataset,model_id)) #indices
full_ground_truth = np.load(experiment_folder + "%s_%s_full_labels.npy"%(sel_dataset,model_id))   #labels


#Load the preprocessed results corresponding to the selected adversarial attacks
results_root = root_path + "adv_attacks/text/analysis/targeted/%s/%s/" % (sel_dataset, model_id)
appendix_param_npy = "%s_targ.npy" % sel_attack


#The following matrices contain information about targeted attacks applied to a set of inputs (row-wise).
#This information is already precomputed for the sake of a more efficient evaluation process.
#The i-th row corresponds to an input sample, and the columns represent the (target) classes.
#Thus, for each row "i" and column "j", the following matrices represent:
# 1) whether it was possible to reach target class y_j from the i-th input (=1) or not (=0).
full_reachability       = np.load(results_root + "reachability_" + appendix_param_npy)
# 2) the amount of distortion required to reach the class y_j from the i-th input.
full_reachability_norms = np.load(results_root + "dist_vec_" + appendix_param_npy)
# 3) the number of steps/"model queries" required to reach the class y_j from the i-th input.
full_reachability_steps = np.load(results_root + "num_steps_vec_" + appendix_param_npy)
# 4) the time required to generate the corresponding attack.
full_reachability_time  = np.load(results_root + "secs_vec_" + appendix_param_npy)


#Now we "nullify" those attacks that surpass any of the thresholds, namely, the maximum
#number of iterations or the maximum distortion allowed.
max_dist_thr = 0.25    #Distortion threshold (\epsilon)
max_iter_thr = np.inf  #Number of iterations (no limit is set)

#Nullify the "reach" of those attacks that exceed the thresholds
for i in range(full_reachability.shape[0]):
    for j in range(num_classes):
        #The norm surpasses the threshold:
        if full_reachability_norms[i,j] > max_dist_thr:
            full_reachability[i,j] = 0
        #The number of steps surpasses the threshold
        if full_reachability_steps[i,j] > max_iter_thr:
            full_reachability[i,j] = 0


#Sanity checks:
assert full_reachability.shape[0] == len(full_filenames)
assert full_reachability.shape[1] == num_classes
#Ensure that the ground-truth class can always be reached
for i in range(full_reachability.shape[0]):
    assert full_reachability[i,full_ground_truth[i]]==1

In [ ]:
#Number of samples per class in the loaded set
N_per_class = [np.sum(full_ground_truth==l) for l in range(len(labels))]
N_per_class = np.array(N_per_class, dtype=np.int16)


#Total number of samples to be considered for the experimental evaluation
N_samp = 1000
#Number of classes per class for the experimental evaluation. In both cases, the proportions are determined
#by the source probability distribution.
N_per_class_train = np.array(p_y_init*N_samp, dtype=np.int16) #to generate the transition matrices
N_per_class_valid = np.array(p_y_init*N_samp, dtype=np.int16) #to evaluate their effectiveness on different inputs

#Sanity checks
assert np.sum(N_per_class_train)==N_samp
assert np.sum(N_per_class_valid)==N_samp
for l in range(len(labels)):
    assert (N_per_class_train[l]+N_per_class_valid[l])<=N_per_class[l]



#Create a hold-out partition for the experiments 
np.random.seed(10)

#Auxiliary mask to assess if a sample has been already selected
samp_visited = [0 for i in range(len(full_filenames))]

#Initialize emtpy lists to store the indices of each partition
train_partition = []
valid_partition = []

for partition in ["train", "valid"]:
    for l in range(len(labels)):
        cont = 0
        #Get the expected number of samples for the current label
        if partition=="train":
            expected_cont = np.copy(N_per_class_train[l])
        else:
            expected_cont = np.copy(N_per_class_valid[l])
        
        #Sample:
        for i in range(len(full_filenames)):
            #Check if the current sample is eligible
            if samp_visited[i]==0 and full_ground_truth[i]==l:
                samp_visited[i]=1 #mark it as selected (unavailable for future usage)
                cont+=1           #increase the counter
                #Add the index to the corresponding partition
                if partition=="train":
                    train_partition.append(i)
                else:
                    valid_partition.append(i)
                    
            #If we already achieved the desired number of samples for this category, exit
            if cont==expected_cont: 
                break
        #Sanity check
        assert cont==expected_cont, "%s, %d %d"%(partition, cont, expected_cont)

#Convert the lists to numpy arrays
train_partition = np.array(train_partition, dtype=np.int16)
valid_partition = np.array(valid_partition, dtype=np.int16)

#Randomly shuffle the order of the indices
np.random.shuffle(train_partition)
np.random.shuffle(valid_partition)

#Sanity checks
assert np.sum(N_per_class_train)==len(train_partition)
assert np.sum(N_per_class_valid)==len(valid_partition)
assert np.sum(samp_visited)==(np.sum(N_per_class_train)+np.sum(N_per_class_valid)), "Wrong sampling size"
for l in range(len(labels)):
    assert np.sum(full_ground_truth[train_partition]==l) == N_per_class_train[l]
    assert np.sum(full_ground_truth[valid_partition]==l) == N_per_class_valid[l]

    
#Split the attack information according to the partitions
#Reach of the attacks:
reachability_train = np.copy(full_reachability[train_partition,:])
reachability_valid = np.copy(full_reachability[valid_partition,:])
#Ground-truth labels:
ground_truth_train = np.copy(full_ground_truth[train_partition])
ground_truth_valid = np.copy(full_ground_truth[valid_partition])


#Sanity checks... (ensure that all are selected, no overlap, etc.)
if len(np.unique(train_partition)) != np.sum(N_per_class_train): sys.exit("Reps. in train partition")
if len(np.unique(valid_partition)) != np.sum(N_per_class_valid): sys.exit("Reps. in valid partition")
if len(np.unique(np.hstack((train_partition,valid_partition)))) != np.sum(N_per_class_train)+np.sum(N_per_class_valid):
    sys.exit("Reps in the merge between train and valid partitions")
#Sanity checks: shape of reachability matrix
if reachability_train.shape[0]     != np.sum(N_per_class_train): sys.exit("Wrong rows in train")
if reachability_valid.shape[0]     != np.sum(N_per_class_valid): sys.exit("Wrong rows in valid")
if reachability_train.shape[1]     != num_classes:                     sys.exit("Wrong cols in train")
if reachability_valid.shape[1]     != num_classes:                     sys.exit("Wrong cols in valid")
#Sanity checks: initial distributions
if len(ground_truth_train)         != np.sum(N_per_class_train): sys.exit("Wrong length in train GT")
if len(ground_truth_valid)         != np.sum(N_per_class_valid): sys.exit("Wrong length in valid GT")
if np.any( [np.sum(ground_truth_train==l)!=N_per_class_train[l] for l in range(num_classes)] ):
    sys.exit("Train sampling did not maintain the initial prob. distr.!")
if np.any( [np.sum(ground_truth_valid==l)!=N_per_class_valid[l] for l in range(num_classes)] ):
    sys.exit("Valid sampling did not maintain the initial prob. distr.!")

In [ ]:
#NUMBER OF SAMPLES TO CONSIDER DURING THE GENERATION OF TARGET DISTRIBUTIONS (TO ESTIMATE THE P-VALUE )
eval_n_samp  = int(np.sum(N_per_class_valid)) #same size that we will encounter in validation

# Sample 'non-significantly different' target distributions 

In [ ]:
np.random.seed(10)

tmp_n_dirichlets   = 1000 #number of distributions to sample
p_y_obj_set_signif = np.zeros((tmp_n_dirichlets, len(labels)))
p_y_obj_set        = np.zeros((tmp_n_dirichlets, len(labels)))

#Sample the random Dirichlet distributions
min_thr_freq = 50 #If a distribution has a frequency lower than this in any category, discard it

for i_p in range(tmp_n_dirichlets):
    cur_sampled_dist = np.copy(np.random.dirichlet([1 for i in range(len(labels))]))
    while np.min(cur_sampled_dist*N_samp)<min_thr_freq:
        cur_sampled_dist = np.copy(np.random.dirichlet([1 for i in range(len(labels))]))
    p_y_obj_set_signif[i_p,:] = np.copy(cur_sampled_dist)
    
    
#Adjust the sampled distributions so that they are not significantly different to the initial distribution    
for i_p in range(p_y_obj_set_signif.shape[0]):
    print(i_p)
    p_y_target_signif = np.copy(p_y_obj_set_signif[i_p,:])
    p_y_target = make_py_target_not_significant(p_y_init, p_y_target_signif,
                                                py_test_tol, py_test_width,
                                                eval_n_samp, labels, selected_stat_test)

    p_y_obj_set[i_p,:] = np.copy(p_y_target) #Save the adjusted distribution
    
    #Sanity checks
    assert evaluate_test(labels, eval_n_samp, p_y_init, p_y_target, selected_stat_test)>(py_test_tol+py_test_width)
    assert np.round(np.min(p_y_target*N_samp))>=min_thr_freq, "Not all the frequencies are above the threshold"

# Execute our methods

In [ ]:
#Checkpoints to see whether during the process the statistical test detects significant label-shifts
checkpoints = np.linspace(100, np.sum(N_per_class_valid), num=10, dtype=np.int16)

In [ ]:
samples_per_class_train = np.copy(N_per_class_train) 
samples_per_class_valid = np.copy(N_per_class_valid)
print("samples_per_class_train:", samples_per_class_train)
print("samples_per_class_valid:", samples_per_class_valid)

In [ ]:
#Method to sample T
get_row_probs_method = 1 #(default)

In [ ]:
np.random.seed(10)

#SUCCES AND FOOLING RATE
success_optimization = np.zeros(len(p_y_obj_set), dtype=int) #succes generating T in the optimization
fooling_rates        = np.zeros(len(p_y_obj_set)) #fooling rate of the attack for the individual samples
#opt_fooling_rate     = np.zeros(len(p_y_obj_set)) #Opt FR that can be obtained in the current valid. set
#KULLBACK-LEIBLER
kl_diver_orig        = np.zeros(len(p_y_obj_set)) #kullback-leibler divergences between original & empirical 
kl_diver_obj         = np.zeros(len(p_y_obj_set)) #kullback-leibler divergence between objective & empirical
kl_diver_orig_obj    = np.zeros(len(p_y_obj_set)) #kullback-leibler divergence between original & empirical
#DISTANCE-BASED METRICS
max_diff_orig        = np.zeros(len(p_y_obj_set)) #Max  diff. between original & empirical
mean_diff_orig       = np.zeros(len(p_y_obj_set)) #Mean diff. between original & empirical
max_diff_obj         = np.zeros(len(p_y_obj_set)) #Max  diff. between objective & empirical
mean_diff_obj        = np.zeros(len(p_y_obj_set)) #Mean diff. between objective & empirical
max_diff_orig_obj    = np.zeros(len(p_y_obj_set)) #Max  diff. between original & objective
mean_diff_orig_obj   = np.zeros(len(p_y_obj_set)) #Mean diff. between original & objective
#CORRELATIONS
spearman_obj         = np.zeros(len(p_y_obj_set)) #Spearman correlation between objective & empirical
pearson_obj          = np.zeros(len(p_y_obj_set)) #Pearson correlation between objective & empirical

# ORIGINAL AND PREDICTED CLASSES
save_orig_classes = np.zeros((len(p_y_obj_set), len(ground_truth_valid)), dtype=int)  # Ground-truth classes of the validation set
save_adv_classes  = np.zeros((len(p_y_obj_set), len(ground_truth_valid)), dtype=int)  # Adversarial classes sampled for the validation set


# PVALUES
#During the validation process (intermediate steps)
pval_orig_checks  = np.zeros((len(p_y_obj_set), len(checkpoints))) #Shift significance between original and "clean" empirical
pval_adv_checks   = np.zeros((len(p_y_obj_set), len(checkpoints))) #Shift significance between original and "adv."  empirical

        
#We also compute beforehand the maximum FR that can be obtained in the valid set.
#We do it now because it only depends on the "training" set, not in the objective prob. distr.
sum_reach_valid_rowise = np.sum(reachability_valid, axis=1) #Sum the number of classes that can be reached from each input
#Count the ratio of cases in which more than one class can be reached (note that the source class can always
#be reached)
opt_fooling_rate = np.sum(sum_reach_valid_rowise>1)/float(reachability_valid.shape[0])


#Sanity checks
if len(sum_reach_valid_rowise)!=reachability_valid.shape[0]: sys.exit("Wrong axis for sum")
if len(sum_reach_valid_rowise)!=np.sum(N_per_class_valid):   sys.exit("Wrong axis for sum")
if np.any(sum_reach_valid_rowise<1):             sys.exit("Zero-row in reachability_valid")
if np.any(np.sum(reachability_train, axis=1)<1): sys.exit("Zero-row in reachability_train")  


    
    
# Compute the auxiliary matrix R 
R = np.zeros((num_classes,num_classes))
for i in range(reachability_train.shape[0]):
    cur_gt_label = ground_truth_train[i] #ground-truth label
    for target_class in range(num_classes):
        if reachability_train[i,target_class]==1: 
            R[cur_gt_label,target_class] += 1

#Sanity checks:
assert np.sum(R)==np.sum(reachability_train)
for i in range(num_classes):
    assert R[i,i] == samples_per_class_train[i]


#Process every p_y_obj in the set
##################################
for i_p in range(len(p_y_obj_set)):
    p_y_obj = p_y_obj_set[i_p]

    #Generate the transition matrix (using Method 3: Element-wise Transformation Method - EWTM)
    T_norm, solver_status = generate_T_method_3(p_y_init, samples_per_class_train, R, p_y_obj, num_classes) 
    
    #Sanity checks
    if solver_status!="Optimal":
        warnings.warn("Not solution found")
        continue

    #Check that results are correct
    T_norm = np.clip(T_norm, a_min=0.0, a_max=1.0) #clip to avoid precision errors
    report_failure = check_results(T_norm, p_y_init, p_y_obj, num_classes, tolerance=1e-5) #check constraints
    if report_failure: sys.exit("Failure reported in the generated T!")


    # Evaluate the effectiveness of the generated transition matrix in another set of inputs
    original_props, \
    adversarial_props, \
    original_classes, \
    predicted_classes = validate_T_matrix(reachability_valid,
                                          ground_truth_valid,
                                          get_row_probs_method,
                                          T_norm,  num_classes)

    #SUCCESS AND FOOLING RATE
    success_optimization[i_p] = 1  #Mark that a solution to the linear program was found (i.e., a matrix T)
    fooling_rates[i_p] = np.sum(original_classes!=predicted_classes)/len(original_classes)

    #DISTANCE-BASED METRICS
    max_diff_orig[i_p]      = np.max( np.abs( np.array(original_props) - np.array(adversarial_props) ))
    mean_diff_orig[i_p]     = np.mean(np.abs( np.array(original_props) - np.array(adversarial_props) ))
    max_diff_obj[i_p]       = np.max( np.abs( np.array(p_y_obj)  - np.array(adversarial_props) ))
    mean_diff_obj[i_p]      = np.mean(np.abs( np.array(p_y_obj)  - np.array(adversarial_props) ))
    max_diff_orig_obj[i_p]  = np.max( np.abs( np.array(p_y_obj)  - np.array(original_props) ))
    mean_diff_orig_obj[i_p] = np.mean(np.abs( np.array(p_y_obj)  - np.array(original_props) ))
    #CORRELATIONS
    spearman_obj[i_p] = sp.stats.spearmanr(adversarial_props,p_y_obj)[0]
    pearson_obj[i_p]  = sp.stats.pearsonr(adversarial_props,p_y_obj)[0]
    #KULLBACK-LEIBLER
    kl_diver_orig[i_p]     = entropy(original_props, adversarial_props)
    kl_diver_obj[i_p]      = entropy(p_y_obj,        adversarial_props)
    kl_diver_orig_obj[i_p] = entropy(original_props, p_y_obj)
    #Sanity check: avoid INF values of the KL because of zeros in the produced distribution
    if np.isinf(kl_diver_orig[i_p]) or np.isinf(kl_diver_obj[i_p]):
        if np.any(adversarial_props==0.0): 
            #Laplace smoothing
            sm_adv_props = np.copy(adversarial_props)
            sm_adv_props = np.array([sm_adv_props[i]*np.sum(N_per_class_valid)+1 for i in range(num_classes)])
            sm_adv_props = np.array([sm_adv_props[i]/np.sum(sm_adv_props) for i in range(num_classes)])
            if np.abs(np.sum(sm_adv_props)-1.0)>1e-8: sys.exit("Check Laplacian smoothing")
            kl_diver_orig[i_p] = entropy(original_props, sm_adv_props)
            kl_diver_obj[i_p]  = entropy(p_y_obj,        sm_adv_props)
            if np.isinf(kl_diver_orig[i_p]) or np.isinf(kl_diver_obj[i_p]): sys.exit("Check KL computation")
        else: sys.exit("Check KL computation (inf/nan but nonzeros)")
            
    # ORIGINAL / ADVERSARIAL CLASSES
    save_orig_classes[i_p,:] = np.copy(original_classes)
    save_adv_classes[i_p, :] = np.copy(predicted_classes)


    #Compute the "intermediate" p-values (i.e., during the attack process)
    for i_check, n_check in enumerate(checkpoints):
        #Select only the first "n_check" samples
        samp_orig = np.copy(original_classes[0:n_check])
        samp_pred = np.copy(predicted_classes[0:n_check])
        assert len(samp_orig)==n_check and len(samp_pred)==n_check, "Wrong sizes" #Sanity check
        #Get the corresponding proportions
        samp_p_y_orig = np.array([np.sum(samp_orig==l) for l in range(len(labels))]) / n_check
        samp_p_y_adv  = np.array([np.sum(samp_pred==l) for l in range(len(labels))]) / n_check
        #Evaluate the statistical test
        pval_orig_checks[i_p,i_check] = evaluate_test(labels, n_check, py_exp=p_y_init, py_obs=samp_p_y_orig, test_method=selected_stat_test)
        pval_adv_checks[i_p, i_check] = evaluate_test(labels, n_check, py_exp=p_y_init, py_obs=samp_p_y_adv,  test_method=selected_stat_test)
        #Sanity checks
        assert np.abs(np.sum(samp_p_y_orig)-1.0)<1e-8 and np.abs(np.sum(samp_p_y_adv)-1.0)<1e-8
        assert np.all(samp_p_y_orig>=0.0) and np.all(samp_p_y_orig<=1.0)
        assert np.all(samp_p_y_adv >=0.0) and np.all(samp_p_y_adv <=1.0)



#If a path was set for save_path, save the results
if not (save_path is None):
    #String to be appended at the end of the filenames, to identify the configuration
    param_appendix = "_eps_%s_init_%d.npy" % (str(max_dist_thr), p_y_init_idx)
    
    # SAVE THE SUCCES INFORMATION AND FOOLING RATES
    np.save(save_path + "success_opt"       + param_appendix, success_optimization)
    np.save(save_path + "fr"                + param_appendix, fooling_rates)
    np.save(save_path + "opt_fr"            + param_appendix, opt_fooling_rate)
    # SAVE THE KULLBACK-LEIBLER INFORMATION
    np.save(save_path + "kl_orig"           + param_appendix, kl_diver_orig)
    np.save(save_path + "kl_obj"            + param_appendix, kl_diver_obj)
    np.save(save_path + "kl_orig_obj"       + param_appendix, kl_diver_orig_obj)
    # SAVE THE DISTANCE BASED METRICS
    np.save(save_path + "max_dif_orig"      + param_appendix, max_diff_orig)
    np.save(save_path + "mean_dif_orig"     + param_appendix, mean_diff_orig)
    np.save(save_path + "max_dif_obj"       + param_appendix, max_diff_obj)
    np.save(save_path + "mean_dif_obj"      + param_appendix, mean_diff_obj)
    np.save(save_path + "max_dif_orig_obj"  + param_appendix, max_diff_orig_obj)
    np.save(save_path + "mean_dif_orig_obj" + param_appendix, mean_diff_orig_obj)
    # SAVE THE CORRELATIONS
    np.save(save_path + "spearman_obj"      + param_appendix, spearman_obj)
    np.save(save_path + "pearson_obj"       + param_appendix, pearson_obj)

    # SAVE THE ORIGINAL / ADVERSARIAL CLASSES
    np.save(save_path + "save_orig_classes" + param_appendix, save_orig_classes)
    np.save(save_path + "save_adv_classes"  + param_appendix, save_adv_classes)

    # SAVE THE PVALUES OF THE STATISTICAL TEST USED BY THE LABEL-SHIFT DETECTOR
    np.save(save_path + "pval_orig_checks"  + param_appendix, pval_orig_checks)
    np.save(save_path + "pval_adv_checks"   + param_appendix, pval_adv_checks)
    np.save(save_path + "checkpoints"       + param_appendix, checkpoints)

    # ASVE THE INITIAL AND SET OF TARGET DISTRIBUTIONS
    np.save(save_path + "p_y_init"          + param_appendix, p_y_init)
    np.save(save_path + "p_y_obj_set"       + param_appendix, p_y_obj_set)
    
    # SAVE THE REMAINING AUXILIARY INFORMATION
    np.save(save_path + "py_tolerance"      + param_appendix, py_test_tol)
    np.save(save_path + "py_tol_width"      + param_appendix, py_test_width)
    print("Results saved")
    
    
print("Job done!")